# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**: Axel Escoto Garcia

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://38f00c0193b4:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/30 03:15:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
# Build schema
# Import your module
from axel2293.spark_utils import SparkUtils

# Agencies, Brands, Cars, Customers, Rentals
schema_agencies = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])
schema_brands = SparkUtils.generate_schema([("brand_id", "int"), ("brand_info", "string")])
schema_cars = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])
schema_customers = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])
schema_rentals = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])

df_agencies = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")
df_brands = spark.read.schema(schema_brands).option("header", True).csv("/opt/spark/work-dir/data/car_service/brands")
df_cars = spark.read.schema(schema_cars).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")
df_customers = spark.read.schema(schema_customers).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers")
df_rentals = spark.read.schema(schema_rentals).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")

df_agencies.show(truncate=False)
df_brands.show(truncate=False)
df_cars.show(truncate=False)
df_customers.show(truncate=False)
df_rentals.show(truncate=False)


+---------+-----------------------------------------------------+
|agency_id|agency_info                                          |
+---------+-----------------------------------------------------+
|1        |{'agency_name': 'NYC Rentals', 'city': 'New York'}   |
|2        |{'agency_name': 'LA Car Rental', 'city': 'Londres'}  |
|3        |{'agency_name': 'Zapopan Auto', 'city': 'Zapopan'}   |
|4        |{'agency_name': 'SF Cars', 'city': 'San Francisco'}  |
|5        |{'agency_name': 'Mexico Cars', 'city': 'Mexico City'}|
+---------+-----------------------------------------------------+

+--------+-----------------------------------------------------+
|brand_id|brand_info                                           |
+--------+-----------------------------------------------------+
|1       |{'brand_name': 'Mercedes-Benz', 'country': 'Germany'}|
|2       |{'brand_name': 'BMW', 'country': 'Germany'}          |
|3       |{'brand_name': 'Audi', 'country': 'Germany'}         |
|4       |{'bra

# Format JSON cols

In [14]:
from pyspark.sql.functions import col, get_json_object

# rental_id|car_name |agency_name |customer_name
# Need columns to do the joins between the tables, we do this with the IDS (car_id, agency_id, customer_id)
df_rentals_f = df_rentals.withColumn("car_id", get_json_object(col("rental_info"), "$.car_id").cast("int")) \
                        .withColumn("agency_id", get_json_object(col("rental_info"), "$.agency_id").cast("int")) \
                        .withColumn("customer_id", get_json_object(col("rental_info"), "$.customer_id").cast("int"))
# Get only the required data (Names)
df_cars_f = df_cars.withColumn("car_name", get_json_object(col("car_info"), "$.car_name")).drop("car_info")
df_agencies_f = df_agencies.withColumn("agency_name", get_json_object(col("agency_info"), "$.agency_name")).drop("agency_info")
df_customers_f = df_customers.withColumn("customer_name", get_json_object(col("customer_info"), "$.customer_name")).drop("customer_info")

df_rentals_f.show()
df_cars_f.show()
df_agencies_f.show()
df_customers_f.show()

+---------+--------------------+------+---------+-----------+
|rental_id|         rental_info|car_id|agency_id|customer_id|
+---------+--------------------+------+---------+-----------+
|    11891|{'car_id': 21, 'c...|    21|        1|         71|
|    11892|{'car_id': 11, 'c...|    11|        2|         52|
|    11893|{'car_id': 22, 'c...|    22|        4|        116|
|    11894|{'car_id': 5, 'cu...|     5|        1|        107|
|    11895|{'car_id': 4, 'cu...|     4|        4|         53|
|    11896|{'car_id': 8, 'cu...|     8|        2|        131|
|    11897|{'car_id': 23, 'c...|    23|        3|         66|
|    11898|{'car_id': 24, 'c...|    24|        4|         60|
|    11899|{'car_id': 27, 'c...|    27|        2|         92|
|    11900|{'car_id': 3, 'cu...|     3|        4|         40|
|    11901|{'car_id': 3, 'cu...|     3|        1|         35|
|    11902|{'car_id': 3, 'cu...|     3|        1|         72|
|    11903|{'car_id': 29, 'c...|    29|        3|         58|
|    119

In [15]:
# Do the joins with the columns we selected and do inner joins (only matching data from both sides)

df_result = df_rentals_f \
                .join(df_cars_f, "car_id", "inner") \
                .join(df_agencies_f, "agency_id", "inner") \
                .join(df_customers_f, "customer_id", "inner") \
                .select("rental_id", "car_name", "agency_name", "customer_name" )

df_result.show()

+---------+--------------------+-------------+---------------+
|rental_id|            car_name|  agency_name|  customer_name|
+---------+--------------------+-------------+---------------+
|    11891|Wallace-Carlson M...|  NYC Rentals| Margaret Jones|
|    11892|Grimes-Green Model 8|LA Car Rental|Albert Williams|
|    11893|Stewart-Allen Mod...|      SF Cars|  Caleb Fleming|
|    11894|  Campos PLC Model 4|  NYC Rentals|  Andrew Butler|
|    11895|  Wagner LLC Model 1|      SF Cars|  Kristin Potts|
|    11896|Jones, Jefferson ...|LA Car Rental|   Jeremy Parks|
|    11897|Lopez and Sons Mo...| Zapopan Auto|    Terry Wells|
|    11898| Salazar Ltd Model 8|      SF Cars|  Marc Williams|
|    11899|Villanueva PLC Mo...|LA Car Rental| Danny Williams|
|    11900|Faulkner-Howard M...|      SF Cars| Eric Owens PhD|
|    11901|Faulkner-Howard M...|  NYC Rentals|    Laura Perry|
|    11902|Faulkner-Howard M...|  NYC Rentals|     Paul Brown|
|    11903|Atkinson Ltd Mode...| Zapopan Auto|Alexa Her

In [4]:
sc.stop()